<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-12 15:11:33
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: 9.99 K (0.08%)
Current PnL: -22.39 L (-15.23%)
CY Booked + Current PnL: -14.12 L (-9.6%)
-------------------
Total profit:  2.13 L
Total loss:  -24.52 L
-------------------
Total Booked + Current PnL: 12.86 L (10.71%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.24%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.87 L (61.79%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.34%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,4.01,218214.0,16734.0,10474.0,2.38,8.31,4.80,13.51,61.0,1.60,1.68,21.95,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.15,H-MC,13.93,160138.0,33938.0,12235.0,-0.50,26.89,7.64,36.59,111.0,2.77,1.23,42.78,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.73,106752.0,2440.0,14209.0,-0.84,2.34,13.31,15.96,11.0,0.17,0.82,19.63,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,92.74,M-SC,10.73,83015.0,-17762.0,17857.0,-0.38,-17.63,21.51,0.09,243.0,-0.99,0.64,9.06,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-25.98,H-SC,9.16,221252.0,28232.0,19802.0,1.08,14.63,8.95,24.89,134.0,1.43,1.70,41.39,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.0,-9.96,H-MC,4.01,196150.0,-878.0,90504.0,-1.39,-0.45,46.14,45.49,103.0,-0.01,1.51,12.54,XY24,BTT,FMCG
92,WIPRO,243.46,420.0,-14.70,M-LC,6.86,149947.0,-998.0,110451.0,0.10,-0.66,73.66,72.51,54.0,-0.01,1.15,5.32,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.87,H-LC,2.00,206158.0,1658.0,39747.0,1.28,0.81,19.28,20.25,10.0,0.04,1.59,9.61,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.58,H-LC,2.52,168558.0,-32828.0,120435.0,-0.41,-16.30,71.45,43.51,27.0,-0.27,1.30,17.33,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.63,183888.0,-16422.0,76405.0,0.82,-8.20,41.55,29.94,17.0,-0.21,1.41,27.04,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-17.13,H-LC,3.77,127553.0,-137873.0,194786.0,-1.34,-51.94,152.71,21.44,8.0,-0.71,0.98,2.93,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-20.39,H-LC,5.40,159049.0,2872.0,23221.0,-0.41,1.84,14.60,16.71,19.0,0.12,1.22,19.44,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.4,-6.39,M-SC,23.46,90164.0,-86651.0,167444.0,4.88,-49.01,185.71,45.70,197.0,-0.52,0.69,4.88,XY24,NTT,TRAVEL
66,SAMMAANCAP,170.35,326.0,-162.50,M-SC,4.65,71682.0,-30528.0,123917.0,3.91,-29.87,172.87,91.37,206.0,-0.25,0.55,16.36,XY25,NTT,FINANCE
81,TITAGARH,1097.23,1548.0,-14.59,H-SC,7.43,139912.0,-52103.0,130986.0,3.01,-27.13,93.62,41.08,156.0,-0.40,1.08,18.42,XY24,NTT,ENGINEERING
6,ASIANTILES,75.41,137.0,6644.44,L-SC,6.80,74018.0,-19792.0,96408.0,2.44,-21.10,130.25,81.67,270.0,-0.21,0.57,43.34,XR,NTT,CERAMICS
68,SBIN,760.30,863.0,-11.34,M-LC,4.01,218214.0,16734.0,10474.0,2.38,8.31,4.80,13.51,61.0,1.60,1.68,21.95,XY25,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.00,-36.72,M-SC,6.18,94595.0,-34075.0,79375.0,-3.69,-26.48,83.91,35.20,209.0,-0.43,0.73,0.00,X40,NTT,FOOTWEAR
87,VAIBHAVGBL,350.21,521.00,41.05,H-SC,8.26,94178.0,-57813.0,131934.0,-1.89,-38.04,140.09,48.77,125.0,-0.44,0.72,8.86,XR,NTT,JEWELLERY
50,KPIGREEN,497.21,731.35,9.02,H-SC,12.86,129982.0,4685.0,54319.0,-1.84,3.74,41.79,47.09,141.0,0.09,1.00,62.68,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.96,M-SC,4.12,92962.0,1464.0,77289.0,-1.78,1.60,83.14,86.07,219.0,0.02,0.72,34.48,XR,NTT,DURABLES
27,DIXON,14367.00,18940.15,-36.71,H-MC,6.46,126640.0,11704.0,24885.0,-1.60,10.18,19.65,31.83,83.0,0.47,0.97,35.76,X40N,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,92.74,M-SC,10.73,83015.0,-17762.0,17857.0,-0.38,-17.63,21.51,0.09,243.0,-0.99,0.64,9.06,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,84.36,H-SC,20.58,116640.0,-25839.0,45361.0,0.96,-18.14,38.89,13.70,163.0,-0.57,0.90,37.67,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.57,H-SC,3.11,114678.0,-29125.0,55871.0,-0.27,-20.25,48.72,18.60,148.0,-0.52,0.88,26.55,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-68.89,H-SC,2.71,214578.0,-55089.0,91432.0,1.45,-20.43,42.61,13.47,142.0,-0.60,1.65,8.53,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,108.15,M-SC,3.40,136574.0,-37985.0,38022.0,2.10,-21.76,27.84,0.02,221.0,-1.00,1.05,18.43,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.69,236418.0,-59516.0,153435.0,-0.32,-20.11,64.90,31.73,2.0,-0.39,1.82,0.93,X40,BTT,IT
45,INFY,1461.46,2275.00,-22.11,H-LC,10.20,254860.0,-6741.0,152355.0,-0.31,-2.58,59.78,55.67,4.0,-0.04,1.96,3.37,X40,BTT,IT
89,VBL,492.64,671.64,-8.90,H-LC,10.60,276666.0,11626.0,84687.0,0.69,4.39,30.61,36.33,5.0,0.14,2.13,18.27,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.71,H-LC,9.97,197866.0,-18653.0,111557.0,0.52,-8.61,56.38,42.91,7.0,-0.17,1.52,3.62,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-17.13,H-LC,3.77,127553.0,-137873.0,194786.0,-1.34,-51.94,152.71,21.44,8.0,-0.71,0.98,2.93,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.15,H-MC,13.93,160138.0,33938.0,12235.0,-0.5,26.89,7.64,36.59,111.0,2.77,1.23,42.78,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.15,H-MC,13.93,160138.0,33938.0,12235.0,-0.50,26.89,7.64,36.59,111.0,2.77,1.23,42.78,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,9.02,H-SC,12.86,129982.0,4685.0,54319.0,-1.84,3.74,41.79,47.09,141.0,0.09,1.00,62.68,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.21,H-MC,7.43,174600.0,9930.0,58037.0,-0.90,6.03,33.24,41.28,91.0,0.17,1.34,14.49,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-7.16,H-SC,3.09,105552.0,3548.0,40374.0,-0.75,3.48,38.25,43.06,123.0,0.09,0.81,21.19,X40N,ATH,MISC
68,SBIN,760.30,863.00,-11.34,M-LC,4.01,218214.0,16734.0,10474.0,2.38,8.31,4.80,13.51,61.0,1.60,1.68,21.95,XY25,NTT,BANKS


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.54,159677.0,8764.0,48542.0,-0.82,5.81,30.40,37.97,92.0,0.18,1.23,32.58,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,9.02,H-SC,12.86,129982.0,4685.0,54319.0,-1.84,3.74,41.79,47.09,141.0,0.09,1.00,62.68,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-52.21,H-SC,12.43,133438.0,10102.0,123657.0,-0.90,8.19,92.67,108.45,119.0,0.08,1.03,32.37,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-44.96,M-SC,4.12,92962.0,1464.0,77289.0,-1.78,1.60,83.14,86.07,219.0,0.02,0.72,34.48,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-14.70,M-LC,6.86,149947.0,-998.0,110451.0,0.10,-0.66,73.66,72.51,54.0,-0.01,1.15,5.32,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.28,69043.0,-44506.0,84557.0,-1.34,-39.20,122.47,35.27,268.0,-0.53,0.53,68.98,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.00,H-SC,16.82,89307.0,-11656.0,106186.0,-0.23,-11.54,118.90,93.63,149.0,-0.11,0.69,33.02,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-19.60,H-SC,15.68,91575.0,-6405.0,28278.0,1.43,-6.54,30.88,22.32,152.0,-0.23,0.70,32.82,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,84.36,H-SC,20.58,116640.0,-25839.0,45361.0,0.96,-18.14,38.89,13.70,163.0,-0.57,0.90,37.67,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,9.02,H-SC,12.86,129982.0,4685.0,54319.0,-1.84,3.74,41.79,47.09,141.0,0.09,1.00,62.68,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-25.98,H-SC,9.16,221252.0,28232.0,19802.0,1.08,14.63,8.95,24.89,134.0,1.43,1.70,41.39,X40N,NTT,HEALTHCARE
70,SHALBY,261.39,327.00,946.25,M-SC,7.36,142245.0,-38898.0,84366.0,1.20,-21.47,59.31,25.10,232.0,-0.46,1.09,14.00,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-8.90,H-LC,10.60,276666.0,11626.0,84687.0,0.69,4.39,30.61,36.33,5.0,0.14,2.13,18.27,X40N,ATH,FMCG
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.54,159677.0,8764.0,48542.0,-0.82,5.81,30.40,37.97,92.0,0.18,1.23,32.58,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-16.58,H-LC,6.39,208358.0,23877.0,46485.0,1.62,12.94,22.31,38.14,35.0,0.51,1.60,24.92,X40N,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,41.17
2,50,70.92


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.54
LC    34.14
MC    26.29
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.06
X40N     15.35
X40      13.37
XY25     10.69
XR        8.97
AR        8.42
OX40N     7.57
X5K       2.24
MH        1.76
X200      1.41
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.78
H-SC    24.61
H-MC    23.33
M-SC    13.47
M-LC     7.26
M-MC     2.62
L-SC     1.46
L-LC     1.10
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.29,-11.01,46.70
IT,11.97,-19.95,79.85
FINANCE,8.33,-20.22,64.70
BANKS,7.69,-14.18,62.67
MISC,5.91,-19.68,88.29
PAINTS,5.68,-18.23,41.53
HEALTHCARE,5.24,3.83,20.45
ELECTRICAL,4.70,-14.87,64.62
AUTO,4.42,-17.52,68.65


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2857001.0,23
XR,1126854.0,13
AR,1115698.0,10
X40,835047.0,10
XY25,660003.0,8
X40N,610066.0,12
OX40N,474494.0,10
SR,197027.0,2
X5K,147496.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2819156.0,28
M-SC,1493309.0,17
H-LC,1384599.0,18
H-MC,1345382.0,18
M-LC,434715.0,5
M-MC,336524.0,2
L-SC,270914.0,3
L-MC,57935.0,1
L-LC,44125.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      935514.0      6
M-SC       XY24      869645.0      7
H-SC       AR        673748.0      5
           XR        532812.0      6
H-LC       X40       521955.0      5
H-MC       XY24      492930.0      5
M-MC       XY24      336524.0      2
H-LC       XY25      258465.0      3
           X40N      239261.0      5
H-MC       X40       233717.0      4
H-LC       AR        232419.0      2
H-SC       X40N      223187.0      4
H-MC       XY25      223022.0      2
H-SC       OX40N     202549.0      4
           SR        197027.0      2
L-SC       XR        180965.0      2
M-LC       XY24      180503.0      2
M-SC       AR        160989.0      2
H-MC       X40N      147618.0      3
M-LC       X5K       133287.0      1
M-SC       OX40N     132449.0      4
           XR        126934.0      2
           XY25      123917.0      1
H-MC       XR        117757.0      1
M-LC       XR        110451.0      1
L-SC       OX40N      89949.0      1
M-SC       X40        79375.0      1
H-LC       X200       76405.0      1
L-MC       XR         57935.0      1
H-SC       MH         54319.0      1
H-MC       OX40N      49547.0      1
           AR         48542.0      1
L-LC       XY25       44125.0      1
H-LC       XY24       41885.0      1
H-MC       MH         32249.0      1
H-LC       X5K        14209.0      1
M-LC       XY25       10474.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
